In [1]:
import numpy as np
import pickle
import torch
import torch.nn.functional as F

import os
import glob

from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer
from tokenizers.pre_tokenizers import Whitespace

from src.model import SimpleBigramModel
from src.preprocessing import preprocess_wikitext
from src.dataloaders import build_loaders
from hparams import Hparams

In [2]:
hparams = Hparams()

In [3]:
np.load(f'{hparams.tokenized_dir}/wikitext_tokenized.npz')

In [4]:
tokenizer, train_loader, _, _ = build_loaders(hparams)

In [5]:
model = SimpleBigramModel(vocab_size = tokenizer.get_vocab_size())
model = model.to('cuda')

In [8]:
sum(p.numel() for p in model.parameters() if p.requires_grad)

900000000

In [6]:
loss_fn = torch.nn.CrossEntropyLoss()
optim = torch.optim.AdamW(model.parameters())

In [7]:
model.train()
for epoch in range(hparams.epochs):
    for x, y in train_loader:
        optim.zero_grad()

        x = x.to('cuda')
        y = y.to('cuda')
        
        logits = model(x)
        B, T, C = logits.shape
        logits = logits.view(B*T, C)
        y = y.view(B*T)

        loss = loss_fn(logits, y)

        loss.backward()
        optim.step()
        print(loss)

    print(loss)



OutOfMemoryError: CUDA out of memory. Tried to allocate 3.35 GiB (GPU 0; 7.79 GiB total capacity; 6.75 GiB already allocated; 235.38 MiB free; 6.75 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF